<a href="https://colab.research.google.com/github/AI4Bharat/indicTrans/blob/main/indicTrans_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a seperate folder to store everything
!mkdir finetuning
%cd finetuning

/content/finetuning


In [ ]:
# clone the repo for running finetuning
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (267/267), done.
remote: Total 398 (delta 231), reused 251 (delta 126), pack-reused 0
Receiving objects: 100% (398/398), 1.41 MiB | 17.84 MiB/s, done.
Resolving deltas: 100% (231/231), done.
/content/finetuning/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 14.30 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.7

In [ ]:
! sudo apt install tree

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (121 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 160772 files and directories currently install

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# download the indictrans model


# downloading the en-indic model
# this will contain:
# en-indic/
# ├── final_bin                          # contains fairseq dictionaries (we will use this to binarize the new finetuning data)
# │   ├── dict.SRC.txt
# │   └── dict.TGT.txt
# ├── model                              # contains model checkpoint(s)
# │   └── checkpoint_best.pt
# └── vocab                              # contains bpes for src and tgt (since we train seperate vocabularies) generated with subword_nmt (we will use this bpes to convert finetuning data to subwords)
#     ├── bpe_codes.32k.SRC
#     ├── bpe_codes.32k.TGT
#     ├── vocab.SRC
#     └── vocab.TGT



!wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
!unzip indic-en.zip

# if you want to finetune indic-en models, use the link below

# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/en-indic.zip
# !unzip en-indic.zip

# if you want to finetune indic-indic models, use the link below

# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
# !unzip m2m.zip



--2021-06-09 18:47:20--  https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551079075 (4.2G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.24G  61.3MB/s    in 56s     

2021-06-09 18:48:16 (77.9 MB/s) - ‘indic-en.zip’ saved [4551079075/4551079075]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/dict.SRC.txt  
   creating: indic-en/model/
  in

In [ ]:
# In this example, we will finetuning on cvit-pib corpus which is part of the WAT2021 training dataset.

# Lets first download the full wat2021 training data (cvit-pib is a part of this big training set)
# ***Note***: See the next section to mine for mining indic to indic data from english centric WAT data. This dataset can be used to finetune indic2indic model
!wget http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_wat_2021.tar.gz
!tar -xzvf indic_wat_2021.tar.gz
# all train sets will now be in wat2021/train
!mv finalrepo wat2021

--2021-06-09 18:50:23--  http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_wat_2021.tar.gz
Resolving lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)... 130.54.208.131
Connecting to lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)|130.54.208.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 777928004 (742M) [application/x-gzip]
Saving to: ‘indic_wat_2021.tar.gz’

indic_wat_2021.tar. 100%[===================>] 741.89M  13.6MB/s    in 57s     

2021-06-09 18:51:20 (13.1 MB/s) - ‘indic_wat_2021.tar.gz’ saved [777928004/777928004]

finalrepo/
finalrepo/README
finalrepo/dev/
finalrepo/dev/dev.mr
finalrepo/dev/dev.kn
finalrepo/dev/dev.gu
finalrepo/dev/dev.ta
finalrepo/dev/dev.bn
finalrepo/dev/dev.pa
finalrepo/dev/dev.ml
finalrepo/dev/dev.or
finalrepo/dev/dev.en
finalrepo/dev/dev.hi
finalrepo/dev/dev.te
finalrepo/train/
finalrepo/train/zeroshotcorpstats
finalrepo/train/opensubtitles/
finalrepo/train/opensubtitles/en-ta/
finalrepo/train/opensubtitles

In [ ]:
# this cell is for mining indic to indic data from a english centric corpus. This data can then be used to our finetune indic2indic model

# Mining Indic to Indic pairs from english centric corpus
# The `extract_non_english_pairs` in `scripts/extract_non_english_pairs.py` can be used to mine indic to indic pairs from english centric corpus.

# As described in the paper (section 2.5) , we use a very strict deduplication criterion to avoid the creation of very similar parallel sentences. 
# For example, if an en sentence is aligned to M hi sentences and N ta sentences, then we would get MN hi-ta pairs. However, these pairs would be very similar and not contribute much to the training process. 
# Hence, we retain only 1 randomly chosen pair out of these MN pairs.

!mkdir wat2021-indic2indic

from indicTrans.scripts.extract_non_english_pairs import extract_non_english_pairs

"""
extract_non_english_pairs(indir, outdir, LANGS)

    Extracts non-english pair parallel corpora
    indir: contains english centric data in the following form:
            - directory named en-xx for language xx
            - each directory contains a train.en and train.xx
    outdir: output directory to store mined data for each pair.
            One directory is created for each pair.
    LANGS: list of languages in the corpus (other than English).
            The language codes must correspond to the ones used in the
            files and directories in indir. Prefarably, sort the languages
            in this list in alphabetic order. outdir will contain data for xx-yy,
            but not for yy-xx, so it will be convenient to have this list in sorted order.
"""
# here we are using three langs to mine bn-hi, hi-gu and gu-bn pairs from wat2021/cvit-pib en-X data
# you should see the following files after running the code below
#  wat2021-indic2indic
#  ├── bn-gu
#  │   ├── train.bn
#  │   └── train.gu
#  ├── bn-hi
#  │   ├── train.bn
#  │   └── train.hi
#  └── hi-gu
#      ├── train.gu
#      └── train.hi

# NOTE: Make sure to dedup the output text files and remove any overlaps with test sets before finetuning
#       Both of the above are implemented in scripts/remove_train_devtest_overlaps.py -> remove_train_devtest_overlaps(train_dir, devtest_dir, many2many=True)

extract_non_english_pairs('wat2021/train/cvit-pib', 'wat2021-indic2indic', ['bn', 'hi', 'gu'])

mkdir: cannot create directory ‘wat2021-indic2indic’: File exists


  0%|          | 0/2 [00:00<?, ?it/s]


bn hi
bn gu


 50%|█████     | 1/2 [03:46<03:46, 226.18s/it]


hi gu


100%|██████████| 2/2 [06:05<00:00, 182.80s/it]


In [ ]:
# wat2021
# ├── dev                    # contains Wat2021 dev data
# │   ├── dev.bn
# │   ├── dev.en
# │   ├── dev.gu
# │   ├── dev.hi
# │   ├── dev.kn
# │   ├── dev.ml
# │   ├── dev.mr
# │   ├── dev.or
# │   ├── dev.pa
# │   ├── dev.ta
# │   └── dev.te
# ├── README
# ├── test                  # contains Wat2021 test data
# │   ├── test.bn
# │   ├── test.en
# │   ├── test.gu
# │   ├── test.hi
# │   ├── test.kn
# │   ├── test.ml
# │   ├── test.mr
# │   ├── test.or
# │   ├── test.pa
# │   ├── test.ta
# │   └── test.te
# └── train                 # contains WAT2021 train data which has lot of corpuses (alt, bible, Jw300, etc)
#     ├── alt/
#     ├── bibleuedin/
#     ├── iitb/
#     ├── jw/
#     ├── mtenglish2odia/
#     ├── nlpc/
#     ├── odiencorp/
#     ├── opensubtitles/
#     ├── pmi/
#     ├── tanzil/
#     ├── ted2020/
#     ├── ufal/
#     ├── urst/
#     ├── wikimatrix/
#     ├── wikititles/
#     └──  cvit-pib         
#         ├── en-bn         # within a train corpus folder the files are arranged in {src_lang}-{tgt_lang}/train.{src_lang}, train.{tgt_lang}
#         │   ├── train.bn
#         │   └── train.en
#         ├── en-gu
#         │   ├── train.en
#         │   └── train.gu
#         ├── en-hi
#         │   ├── train.en
#         │   └── train.hi
#         ├── en-ml
#         │   ├── train.en
#         │   └── train.ml
#         ├── en-mr
#         │   ├── train.en
#         │   └── train.mr
#         ├── en-or
#         │   ├── train.en
#         │   └── train.or
#         ├── en-pa
#         │   ├── train.en
#         │   └── train.pa
#         ├── en-ta
#         │   ├── train.en
#         │   └── train.ta
#         └── en-te
#             ├── train.en
#             └── train.te



# instead of using all the data for this example, we will mainly use the cvit-pib corpus from wat2021 train set
# for dev and test set, we will use the dev and test provided by wat2021

# In case, you want to finetune on all these corpuses, you would need to merge all the training data into one folder and remove duplicate train sentence pairs.
# To do this, refer to this gist: https://gist.github.com/gowtham1997/2524f8e9559cff586d1f935e621fc598


# copy everything to a dataset folder
!mkdir -p dataset/train
! cp -r wat2021/train/cvit-pib/* dataset/train
! cp -r wat2021/dev dataset
! cp -r wat2021/test dataset


# lets cd to indicTrans
%cd indicTrans

/content/finetuning/indicTrans


In [ ]:
%%shell

exp_dir=../dataset
src_lang=en
tgt_lang=indic

# change this to indic-en, if you have downloaded the indic-en dir or m2m if you have downloaded the indic2indic model
download_dir=../en-indic

train_data_dir=$exp_dir/train
dev_data_dir=$exp_dir/dev
test_data_dir=$exp_dir/test
echo $exp_dir



../dataset


In [ ]:
# all the data preparation happens in this cell
%%shell

exp_dir=../dataset
src_lang=en
tgt_lang=indic

# change this to indic-en, if you have downloaded the indic-en dir or m2m if you have downloaded the indic2indic model
download_dir=../en-indic

train_data_dir=$exp_dir/train
dev_data_dir=$exp_dir/dev
test_data_dir=$exp_dir/test


echo "Running experiment ${exp_dir} on ${src_lang} to ${tgt_lang}"


train_processed_dir=$exp_dir/data
devtest_processed_dir=$exp_dir/data

out_data_dir=$exp_dir/final_bin

mkdir -p $train_processed_dir
mkdir -p $devtest_processed_dir
mkdir -p $out_data_dir

# indic languages.
# cvit-pib corpus does not have as (assamese) and kn (kannada), hence its not part of this list
langs=(bn hi gu ml mr or pa ta te)

for lang in ${langs[@]};do
	if [ $src_lang == en ]; then
		tgt_lang=$lang
	else
		src_lang=$lang
	fi

	train_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
	devtest_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
	mkdir -p $train_norm_dir
	mkdir -p $devtest_norm_dir


    # preprocessing pretokenizes the input (we use moses tokenizer for en and indicnlp lib for indic languages)
    # after pretokenization, we use indicnlp to transliterate all the indic data to devnagiri script

	# train preprocessing
	train_infname_src=$train_data_dir/en-${lang}/train.$src_lang
	train_infname_tgt=$train_data_dir/en-${lang}/train.$tgt_lang
	train_outfname_src=$train_norm_dir/train.$src_lang
	train_outfname_tgt=$train_norm_dir/train.$tgt_lang
	echo "Applying normalization and script conversion for train $lang"
	input_size=`python scripts/preprocess_translate.py $train_infname_src $train_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $train_infname_tgt $train_outfname_tgt $tgt_lang true`
	echo "Number of sentences in train $lang: $input_size"

	# dev preprocessing
	dev_infname_src=$dev_data_dir/dev.$src_lang
	dev_infname_tgt=$dev_data_dir/dev.$tgt_lang
	dev_outfname_src=$devtest_norm_dir/dev.$src_lang
	dev_outfname_tgt=$devtest_norm_dir/dev.$tgt_lang
	echo "Applying normalization and script conversion for dev $lang"
	input_size=`python scripts/preprocess_translate.py $dev_infname_src $dev_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $dev_infname_tgt $dev_outfname_tgt $tgt_lang true`
	echo "Number of sentences in dev $lang: $input_size"

	# test preprocessing
	test_infname_src=$test_data_dir/test.$src_lang
	test_infname_tgt=$test_data_dir/test.$tgt_lang
	test_outfname_src=$devtest_norm_dir/test.$src_lang
	test_outfname_tgt=$devtest_norm_dir/test.$tgt_lang
	echo "Applying normalization and script conversion for test $lang"
	input_size=`python scripts/preprocess_translate.py $test_infname_src $test_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $test_infname_tgt $test_outfname_tgt $tgt_lang true`
	echo "Number of sentences in test $lang: $input_size"
done




# Now that we have preprocessed all the data, we can now merge these different text files into one
# ie. for en-as, we have train.en and corresponding train.as, similarly for en-bn, we have train.en and corresponding train.bn
# now we will concatenate all this into en-X where train.SRC will have all the en (src) training data and train.TGT will have all the concatenated indic lang data

python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'train'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'dev'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'test'

# use the vocab from downloaded dir
cp -r $download_dir/vocab $exp_dir


echo "Applying bpe to the new finetuning data"
bash apply_single_bpe_traindevtest_notag.sh $exp_dir

mkdir -p $exp_dir/final

# We also add special tags to indicate the source and target language in the inputs
#  Eg: to translate a sentence from english to hindi , the input would be   __src__en__   __tgt__hi__ <en bpe tokens>

echo "Adding language tags"
python scripts/add_joint_tags_translate.py $exp_dir 'train'
python scripts/add_joint_tags_translate.py $exp_dir 'dev'
python scripts/add_joint_tags_translate.py $exp_dir 'test'



data_dir=$exp_dir/final
out_data_dir=$exp_dir/final_bin

rm -rf $out_data_dir

# binarizing the new data (train, dev and test) using dictionary from the download dir

 num_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`

data_dir=$exp_dir/final
out_data_dir=$exp_dir/final_bin

# rm -rf $out_data_dir

echo "Binarizing data. This will take some time depending on the size of finetuning data"
fairseq-preprocess --source-lang SRC --target-lang TGT \
 --trainpref $data_dir/train --validpref $data_dir/dev --testpref $data_dir/test \
 --destdir $out_data_dir --workers $num_workers \
 --srcdict $download_dir/final_bin/dict.SRC.txt --tgtdict $download_dir/final_bin/dict.TGT.txt --thresholdtgt 5 --thresholdsrc 5  

Running experiment ../dataset on en to indic
Applying normalization and script conversion for train bn
100% 91985/91985 [00:25<00:00, 3582.55it/s]
100% 91985/91985 [00:14<00:00, 6330.85it/s]
Number of sentences in train bn: 91985
Applying normalization and script conversion for dev bn
100% 1000/1000 [00:00<00:00, 1593.70it/s]
100% 1000/1000 [00:00<00:00, 7232.26it/s]
Number of sentences in dev bn: 1000
Applying normalization and script conversion for test bn
100% 2390/2390 [00:00<00:00, 2874.03it/s]
100% 2390/2390 [00:00<00:00, 6727.65it/s]
Number of sentences in test bn: 2390
Applying normalization and script conversion for train hi
100% 266545/266545 [01:15<00:00, 3546.17it/s]
100% 266545/266545 [00:45<00:00, 5913.09it/s]
Number of sentences in train hi: 266545
Applying normalization and script conversion for dev hi
100% 1000/1000 [00:00<00:00, 1666.49it/s]
100% 1000/1000 [00:00<00:00, 5857.08it/s]
Number of sentences in dev hi: 1000
Applying normalization and script conversion for t

In [ ]:
# Finetuning the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-update=1000     -> for this example, to demonstrate how to finetune we are only training for 1000 steps. You should increase this when finetuning
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --restore-file        -> reload the pretrained checkpoint and start training from here (change this path for indic-en. Currently its is set to en-indic)
# --reset-*             -> reset and not use lr scheduler, dataloader, optimizer etc of the older checkpoint
# --max_tokns           -> this is max tokens per batch


!( fairseq-train ../dataset/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=1000 \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 4000 \
--dropout 0.2 \
--tensorboard-logdir ../dataset/tensorboard-wandb \
--save-dir ../dataset/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 256 \
--lr 3e-5 \
--restore-file ../en-indic/model/checkpoint_best.pt \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

2021-05-09 14:29:11 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': '../dataset/tensorboard-wandb', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': 'model_configs', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training

In [ ]:
# To test the models after training, you can use joint_translate.sh



# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to hindi
!bash joint_translate.sh $exp_dir/test/test.en en_hi_outputs.txt 'en' 'hi' $exp_dir

In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language

bash compute_bleu.sh en_hi_outputs.txt $exp_dir/test/test.hi 'en' 'hi'
